# Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import request
import json


In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
'''
url = "https://www.transfermarkt.co.uk/spieler-statistik/wertvollstespieler/marktwertetop"
res = requests.get(url, headers=headers)
page_obj = BeautifulSoup(res.content, 'html.parser')
name_lst = [] # List that will receive all the players names
value_lst = []

#html_names = page_obj.find_all("td",{"class":"hauptlink"})

html_names =page_obj.find_all(lambda tag: tag.name == 'td' and tag.get('class') == ['hauptlink'])
# Now we will get only the names of all players
for name in html_names:
    name_lst.append(name.text)
print(name_lst)

html_values =page_obj.find_all(lambda tag: tag.name == 'td' and tag.get('class') == ['rechts', 'hauptlink'])

# Now we will get only the value of all players
for value in html_values:
    value_lst.append(value.text.replace(u'\xa0', u' '))
print(value_lst)'''

['Kylian Mbappé', 'Erling Haaland', 'Harry Kane', 'Jack Grealish', 'Mohamed Salah', 'Romelu Lukaku', 'Kevin De Bruyne', 'Neymar', 'Jadon Sancho', 'Frenkie de Jong', 'Bruno Fernandes', 'Joshua Kimmich', 'Raheem Sterling', 'Marcus Rashford', 'Sadio Mané', 'Heung-min Son', 'Pedri', 'Phil Foden', 'Lautaro Martínez', 'Marcos Llorente', 'Lionel Messi', 'Mason Mount', 'Trent Alexander-Arnold', 'Rúben Dias', 'Marquinhos']
['£144.00m ', '£135.00m ', '£108.00m ', '£90.00m ', '£90.00m ', '£90.00m ', '£90.00m ', '£90.00m ', '£81.00m ', '£81.00m ', '£81.00m ', '£81.00m ', '£81.00m ', '£76.50m ', '£76.50m ', '£76.50m ', '£72.00m ', '£72.00m ', '£72.00m ', '£72.00m ', '£72.00m ', '£67.50m ', '£67.50m ', '£67.50m ', '£67.50m ']


# Mapping from two sources

In [3]:
mapping ="https://raw.githubusercontent.com/JaseZiv/worldfootballR_data/master/raw-data/fbref-tm-player-mapping/output/fbref_to_tm_mapping.csv"
df= pd.read_csv(mapping,encoding='latin-1')
df

,PlayerFBref,UrlFBref,UrlTmarkt,TmPos
0,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,Centre-Forward
1,Aaron Cresswell,https://fbref.com/en/players/4f974391/Aaron-Cr...,https://www.transfermarkt.com/aaron-cresswell/...,Left-Back
2,AarÃ³n Escandell,https://fbref.com/en/players/67669ce7/Aaron-Es...,https://www.transfermarkt.com/aaron-escandell/...,Goalkeeper
3,Aaron Herzog,https://fbref.com/en/players/565c3fe4/Aaron-He...,https://www.transfermarkt.com/aaron-herzog/pro...,Attacking Midfield
4,Aaron Hickey,https://fbref.com/en/players/1780bb4a/Aaron-Hi...,https://www.transfermarkt.com/aaron-hickey/pro...,Left-Back
...,...,...,...,...
6295,Zinho Vanheusden,https://fbref.com/en/players/afbf187b/Zinho-Va...,https://www.transfermarkt.com/zinho-vanheusden...,Centre-Back
6296,Zlatan IbrahimoviÄ,https://fbref.com/en/players/4cde5509/Zlatan-I...,https://www.transfermarkt.com/zlatan-ibrahimov...,Centre-Forward
6297,Zlatko JunuzoviÄ,https://fbref.com/en/players/dba67bc4/Zlatko-J...,https://www.transfermarkt.com/zlatko-junuzovic...,Central Midfield
6298,Zouhair Feddal,https://fbref.com/en/players/89151b16/Zouhair-...,https://www.transfermarkt.com/zouhair-feddal/p...,Centre-Back


# Define function to scrape from FBreg

In [44]:

fbref_rel = "https://fbref.com/en/players/27c01749/scout/365_euro/Aaron-Connolly-Scouting-Report"

def scrap_fbref(url):
    attr_dict = {}
    res = requests.get(url, headers=headers)
    page_obj = BeautifulSoup(res.content, 'html.parser')
    tb = page_obj.find('tbody')  
    try:
        tr = tb.find_all('tr')
        for each_tr in tr:
            if not each_tr.has_attr('class'):
                th = each_tr.find('th')
                th_text = th.text
                if  th_text is not "":
                    stat = each_tr.find('td')
                    attr_dict[th.text] = stat.text
        return attr_dict
    except: 
        return {}

player_attr_lst = scrap_fbref(fbref_rel)
player_attr_lst

{'Goals': '0.19',
 'Assists': '0.00',
 'Non-Penalty Goals': '0.19',
 'Penalty Kicks Made': '0.00',
 'Penalty Kicks Attempted': '0.00',
 'Yellow Cards': '0.00',
 'Red Cards': '0.00',
 'xG': '0.54',
 'npxG': '0.54',
 'xA': '0.08',
 'npxG+xA': '0.61',
 'Shots Total': '3.06',
 'Shots on target': '0.96',
 'Shots on target %': '31.3%',
 'Goals/Shot': '0.06',
 'Goals/Shot on target': '0.20',
 'Average Shot Distance': '13.40',
 'Shots from free kicks': '0.00',
 'npxG/Sh': '0.17',
 'Goals - xG': '-0.34',
 'Non-Penalty Goals - npxG': '-0.34',
 'Passes Completed': '8.23',
 'Passes Attempted': '10.53',
 'Pass Completion %': '78.2%',
 'Total Passing Distance': '113.94',
 'Progressive Passing Distance': '8.81',
 'Passes Completed (Short)': '5.17',
 'Passes Attempted (Short)': '6.70',
 'Pass Completion % (Short)': '77.1%',
 'Passes Completed (Medium)': '2.68',
 'Passes Attempted (Medium)': '3.06',
 'Pass Completion % (Medium)': '87.5%',
 'Passes Completed (Long)': '0.19',
 'Passes Attempted (Long)': 

# Define function to scrape from tmarket

In [47]:
tmark_url = "https://www.transfermarkt.com/aaron-connolly/profil/spieler/434207"
def scrap_tmark(url):
    res = requests.get(url, headers=headers)
    page_obj = BeautifulSoup(res.content, 'html.parser')
    try:
        price =page_obj.find(lambda tag: tag.name == 'span' and tag.get('class') == ['waehrung'])
        return price.parent.text.split('m')[0]
    except:
        return None
scrap_tmark(tmark_url)

'€7.00'

# Change the url of the fbref to scrape more detail

In [32]:
url = "https://fbref.com/en/players/27c01749/Aaron-Connolly"
def change_url(url):
    split=url.split("/")
    split[-1] = split[-1] + "-Scouting-Report"
    split.insert(-1,'scout')
    split.insert(-1,'365_euro')
    concat = "/".join(split)
    print(concat)
    return concat

In [41]:
def get_record(name,fbref_rel,tmark_url):
    record_dict ={}
    record_dict['Name'] = name
    fbref_rel=change_url(fbref_rel)
    attr_dict = scrap_fbref(fbref_rel)
    record_dict.update(attr_dict)
    value = scrap_tmark(tmark_url)
    record_dict['Market value']=value
    return record_dict

SyntaxError: invalid syntax (<ipython-input-41-da508db03205>, line 10)

# Loop through all player and scraipe from 2 sources (2 urls)

In [48]:
#result = [get_record(row[0],row[1],row[2]) for row in zip(df['PlayerFBref'],df['UrlFBref'], df['UrlTmarkt']) if row[1] == "https://fbref.com/en/players/27c01749/Aaron-Connolly"]
result = [get_record(row[0],row[1],row[2]) for row in zip(df['PlayerFBref'],df['UrlFBref'], df['UrlTmarkt'])]
result

https://fbref.com/en/players/27c01749/scout/365_euro/Aaron-Connolly-Scouting-Report
https://fbref.com/en/players/4f974391/scout/365_euro/Aaron-Cresswell-Scouting-Report
https://fbref.com/en/players/67669ce7/scout/365_euro/Aaron-Escandell-Scouting-Report
https://fbref.com/en/players/565c3fe4/scout/365_euro/Aaron-Herzog-Scouting-Report
https://fbref.com/en/players/1780bb4a/scout/365_euro/Aaron-Hickey-Scouting-Report
https://fbref.com/en/players/5f9f0531/scout/365_euro/Aaron-Hunt-Scouting-Report
https://fbref.com/en/players/54d161a7/scout/365_euro/Aaron-Kamardin-Scouting-Report
https://fbref.com/en/players/2ff964a0/scout/365_euro/Aaron-Lennon-Scouting-Report
https://fbref.com/en/players/29ef0f49/scout/365_euro/Aaron-Leya-Iseka-Scouting-Report
https://fbref.com/en/players/2f3e911a/scout/365_euro/Aaron-Martin-Scouting-Report
https://fbref.com/en/players/47b7e3af/scout/365_euro/Aaron-Mooy-Scouting-Report
https://fbref.com/en/players/466fb2c5/scout/365_euro/Aaron-Ramsdale-Scouting-Report
http

https://fbref.com/en/players/58a3a572/scout/365_euro/Adrian-Guerrero-Scouting-Report
https://fbref.com/en/players/88a359d4/scout/365_euro/Adrian-Lopez-Scouting-Report
https://fbref.com/en/players/c58ebf64/scout/365_euro/Adrian-Lopez-Scouting-Report
https://fbref.com/en/players/8aa5f52c/scout/365_euro/Adrian-Mariappa-Scouting-Report
https://fbref.com/en/players/b7f2edff/scout/365_euro/Adrian-Marin-Scouting-Report
https://fbref.com/en/players/89c870fb/scout/365_euro/Adrian-Oeynhausen-Scouting-Report
https://fbref.com/en/players/c693cf7c/scout/365_euro/Adrian-Ramos-Scouting-Report
https://fbref.com/en/players/22c77851/scout/365_euro/Adrian-Sahibeddine-Scouting-Report
https://fbref.com/en/players/f02dde20/scout/365_euro/Adrian-Semper-Scouting-Report
https://fbref.com/en/players/2a5fab54/scout/365_euro/Adrian-Stanilewicz-Scouting-Report
https://fbref.com/en/players/4468b31a/scout/365_euro/Adrian-Stoian-Scouting-Report
https://fbref.com/en/players/1fc9a88f/scout/365_euro/Adrian-Tellado-Scout

https://fbref.com/en/players/d5127c3f/scout/365_euro/Aleksandar-Katai-Scouting-Report
https://fbref.com/en/players/eda40760/scout/365_euro/Aleksandar-Kolarov-Scouting-Report
https://fbref.com/en/players/3925dbd6/scout/365_euro/Aleksandar-Mitrovic-Scouting-Report
https://fbref.com/en/players/09507ffa/scout/365_euro/Aleksandar-Radovanovic-Scouting-Report
https://fbref.com/en/players/a35134ae/scout/365_euro/Aleksandar-Sedlar-Scouting-Report
https://fbref.com/en/players/c15d1af9/scout/365_euro/Aleksandar-Tonev-Scouting-Report
https://fbref.com/en/players/84bba8b6/scout/365_euro/Aleksandar-Trajkovski-Scouting-Report
https://fbref.com/en/players/59f2d958/scout/365_euro/Aleksander-Buksa-Scouting-Report
https://fbref.com/en/players/3708e9fa/scout/365_euro/Aleksandr-Golovin-Scouting-Report
https://fbref.com/en/players/fec5accc/scout/365_euro/Aleksandr-Kokorin-Scouting-Report
https://fbref.com/en/players/a9aa0d1d/scout/365_euro/Aleksei-Miranchuk-Scouting-Report
https://fbref.com/en/players/a6b9b

https://fbref.com/en/players/0d55d942/scout/365_euro/Alexander-Schwolow-Scouting-Report
https://fbref.com/en/players/e61d33f1/scout/365_euro/Alexander-Soderlund-Scouting-Report
https://fbref.com/en/players/e92cd3f7/scout/365_euro/Alexander-Sorloth-Scouting-Report
https://fbref.com/en/players/99cf1943/scout/365_euro/Alexander-Stolz-Scouting-Report
https://fbref.com/en/players/75566759/scout/365_euro/Alexander-Tettey-Scouting-Report
https://fbref.com/en/players/eeeca247/scout/365_euro/Alexandre-Conceicao-Scouting-Report
https://fbref.com/en/players/7c39363f/scout/365_euro/Alexandre-Jankewitz-Scouting-Report
https://fbref.com/en/players/9dbb75ca/scout/365_euro/Alexandre-Lacazette-Scouting-Report
https://fbref.com/en/players/ae6941ec/scout/365_euro/Alexandre-Lauray-Scouting-Report
https://fbref.com/en/players/96fcd380/scout/365_euro/Alexandre-Letellier-Scouting-Report
https://fbref.com/en/players/e067808c/scout/365_euro/Alexandre-Mendy-Scouting-Report
https://fbref.com/en/players/c977ef13/

https://fbref.com/en/players/ced1d98f/scout/365_euro/Amine-Bassi-Scouting-Report
https://fbref.com/en/players/aad56ca3/scout/365_euro/Amine-Gouiri-Scouting-Report
https://fbref.com/en/players/e7d59f4f/scout/365_euro/Amine-Harit-Scouting-Report
https://fbref.com/en/players/1c08a3c8/scout/365_euro/Amir-Abrashi-Scouting-Report
https://fbref.com/en/players/e621fe01/scout/365_euro/Amir-Adouyev-Scouting-Report
https://fbref.com/en/players/7af1baf2/scout/365_euro/Amir-Arli-Scouting-Report
https://fbref.com/en/players/652c00db/scout/365_euro/Amir-Feratovic-Scouting-Report
https://fbref.com/en/players/4b3e1a38/scout/365_euro/Amir-Rrahmani-Scouting-Report
https://fbref.com/en/players/45d98427/scout/365_euro/Amjhad-Nazih-Scouting-Report
https://fbref.com/en/players/5d2b36a4/scout/365_euro/Amores-Scouting-Report
https://fbref.com/en/players/526ea7cc/scout/365_euro/Amos-Pieper-Scouting-Report
https://fbref.com/en/players/850c0a0d/scout/365_euro/Anaitz-Arbilla-Scouting-Report
https://fbref.com/en/pl

...

https://fbref.com/en/players/97b87472/scout/365_euro/Tomas-Hubocan-Scouting-Report
https://fbref.com/en/players/dc366c36/scout/365_euro/Tomas-Koubek-Scouting-Report
https://fbref.com/en/players/3ba95f20/scout/365_euro/Tomas-Ostrak-Scouting-Report
https://fbref.com/en/players/e54bd0a2/scout/365_euro/Tomas-Pina-Isla-Scouting-Report
https://fbref.com/en/players/f1cfc209/scout/365_euro/Tomas-Rincon-Scouting-Report
https://fbref.com/en/players/6613c819/scout/365_euro/Tomas-Soucek-Scouting-Report
https://fbref.com/en/players/0dce3356/scout/365_euro/Tomas-Tavares-Scouting-Report
https://fbref.com/en/players/7b456af7/scout/365_euro/Tomas-Vaclik-Scouting-Report
https://fbref.com/en/players/6f96667d/scout/365_euro/Tomasz-Kucz-Scouting-Report
https://fbref.com/en/players/27d7453c/scout/365_euro/Tomer-Hemed-Scouting-Report
https://fbref.com/en/players/a5055b1b/scout/365_euro/Tomislav-Gomelt-Scouting-Report
https://fbref.com/en/players/3cd3b384/scout/365_euro/Tommaso-Augello-Scouting-Report
https:/

https://fbref.com/en/players/be32269e/scout/365_euro/Varazdat-Haroyan-Scouting-Report
https://fbref.com/en/players/a9ca50f5/scout/365_euro/Vasco-Oliveira-Scouting-Report
https://fbref.com/en/players/339c6ed4/scout/365_euro/Vasco-Regini-Scouting-Report
https://fbref.com/en/players/75422cbf/scout/365_euro/Vasileios-Pavlidis-Scouting-Report
https://fbref.com/en/players/8279c033/scout/365_euro/Vasilije-Janjicic-Scouting-Report
https://fbref.com/en/players/b4a3d61a/scout/365_euro/Vasilios-Zagaritis-Scouting-Report
https://fbref.com/en/players/8ea2c099/scout/365_euro/Vasilis-Torosidis-Scouting-Report
https://fbref.com/en/players/591477c9/scout/365_euro/Vassilis-Lambropoulos-Scouting-Report
https://fbref.com/en/players/29e48848/scout/365_euro/Vasyl-Kravets-Scouting-Report
https://fbref.com/en/players/c3fd9887/scout/365_euro/Vedad-Ibisevic-Scouting-Report
https://fbref.com/en/players/0cadc46d/scout/365_euro/Vedat-Muriqi-Scouting-Report
https://fbref.com/en/players/f972b073/scout/365_euro/Verza

https://fbref.com/en/players/93b8f9f4/scout/365_euro/Warren-Tchimbembe-Scouting-Report
https://fbref.com/en/players/c149016b/scout/365_euro/Wataru-Endo-Scouting-Report
https://fbref.com/en/players/76eca7e2/scout/365_euro/Wayne-Hennessey-Scouting-Report
https://fbref.com/en/players/f07be45a/scout/365_euro/Wayne-Rooney-Scouting-Report
https://fbref.com/en/players/1d387eee/scout/365_euro/Wayne-Routledge-Scouting-Report
https://fbref.com/en/players/f9acca3a/scout/365_euro/Wendell-Scouting-Report
https://fbref.com/en/players/96cf7b61/scout/365_euro/Wes-Foderingham-Scouting-Report
https://fbref.com/en/players/80c31c97/scout/365_euro/Wes-Morgan-Scouting-Report
https://fbref.com/en/players/7f7ad37a/scout/365_euro/Wesley-Scouting-Report
https://fbref.com/en/players/132a82f1/scout/365_euro/Wesley-Fofana-Scouting-Report
https://fbref.com/en/players/875898c7/scout/365_euro/Wesley-Hoedt-Scouting-Report
https://fbref.com/en/players/6e5b5a45/scout/365_euro/Wesley-Lautoa-Scouting-Report
https://fbref.

https://fbref.com/en/players/f6798fc3/scout/365_euro/Yassine-Bounou-Scouting-Report
https://fbref.com/en/players/0049d544/scout/365_euro/Yassine-El-Ghanassy-Scouting-Report
https://fbref.com/en/players/11ced928/scout/365_euro/Yaya-Sanogo-Scouting-Report
https://fbref.com/en/players/cb4e852b/scout/365_euro/Yaya-Soumare-Scouting-Report
https://fbref.com/en/players/c204f844/scout/365_euro/Yaya-Toure-Scouting-Report
https://fbref.com/en/players/f95f49d2/scout/365_euro/Yayah-Kallon-Scouting-Report
https://fbref.com/en/players/eedea60b/scout/365_euro/Yehvann-Diouf-Scouting-Report
https://fbref.com/en/players/813aae2f/scout/365_euro/Yeni-Ngbakoto-Scouting-Report
https://fbref.com/en/players/3ecc11d8/scout/365_euro/Yeray-Alvarez-Scouting-Report
https://fbref.com/en/players/540ec57b/scout/365_euro/Yeremi-Pino-Scouting-Report
https://fbref.com/en/players/6d5701f2/scout/365_euro/Yerry-Mina-Scouting-Report
https://fbref.com/en/players/1e159d70/scout/365_euro/Yerson-Mosquera-Scouting-Report
https:/

[{'Name': 'Aaron Connolly',
  'Goals': '0.19',
  'Assists': '0.00',
  'Non-Penalty Goals': '0.19',
  'Penalty Kicks Made': '0.00',
  'Penalty Kicks Attempted': '0.00',
  'Yellow Cards': '0.00',
  'Red Cards': '0.00',
  'xG': '0.54',
  'npxG': '0.54',
  'xA': '0.08',
  'npxG+xA': '0.61',
  'Shots Total': '3.06',
  'Shots on target': '0.96',
  'Shots on target %': '31.3%',
  'Goals/Shot': '0.06',
  'Goals/Shot on target': '0.20',
  'Average Shot Distance': '13.40',
  'Shots from free kicks': '0.00',
  'npxG/Sh': '0.17',
  'Goals - xG': '-0.34',
  'Non-Penalty Goals - npxG': '-0.34',
  'Passes Completed': '8.23',
  'Passes Attempted': '10.53',
  'Pass Completion %': '78.2%',
  'Total Passing Distance': '113.94',
  'Progressive Passing Distance': '8.81',
  'Passes Completed (Short)': '5.17',
  'Passes Attempted (Short)': '6.70',
  'Pass Completion % (Short)': '77.1%',
  'Passes Completed (Medium)': '2.68',
  'Passes Attempted (Medium)': '3.06',
  'Pass Completion % (Medium)': '87.5%',
  'P

In [112]:
my_df = pd.DataFrame(result)
my_df.to_csv(r'hi.csv')
my_df = pd.read_csv (r'hi.csv', index_col=[0])
my_df

,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,...,Launch%,Average Pass Length,Goal Kicks,Launch% (Goal Kicks),Avg. Length of Goal Kicks,Crosses Faced,Crosses Stopped,Crosses Stopped %,Def. Actions Outside Pen. Area,Avg. Distance of Def. Actions
0,Aaron Connolly,0.19,0.0,0.19,0.00,0.00,0.00,0.00,0.54,0.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Cresswell,0.02,0.2,0.02,0.00,0.00,0.07,0.00,0.03,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AarÃ³n Escandell,NaN,NaN,NaN,NaN,0.22,NaN,NaN,NaN,NaN,...,69.4%,49.7,8.63,81.0%,59.2,8.85,0.66,7.4%,2.29,16.8
3,Aaron Herzog,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aaron Hickey,0.21,0.0,0.21,0.00,0.00,0.28,0.07,0.03,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,Zinho Vanheusden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6296,Zlatan IbrahimoviÄ,0.57,0.1,0.52,0.05,0.10,0.16,0.05,0.65,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6297,Zlatko JunuzoviÄ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6298,Zouhair Feddal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
def get_tmark_extra(url):
    res = requests.get(url, headers=headers)
    page_obj = BeautifulSoup(res.content, 'html.parser')
    try:
        age = page_obj.find('span',string='Age:').findNext('span')
        return age.text
    except:
        return None
try_url = "https://www.transfermarkt.com/aaron-escandell/profil/spieler/284430"
get_tmark_extra(try_url)

'26'

# Get extra bio from tmark

In [93]:
def get_extra_bio(tmark_url,pos):
    extra_attr_dict = {}
    extra_attr_dict['Position'] = pos
    age = get_tmark_extra(tmark_url)
    extra_attr_dict['Age'] = age
    return extra_attr_dict


In [94]:
#result = [get_extra_bio(row[0],row[1]) for row in zip(df['UrlTmarkt'],df['TmPos'])]

extra_attr_lst = []
for index,row in df.iterrows():
    each_attr_dict=get_extra_bio(row[2],row[3])
    extra_attr_lst.append(each_attr_dict)
    print(each_attr_dict)

{'Position': 'Centre-Forward', 'Age': '21'}
{'Position': 'Left-Back', 'Age': '31'}
{'Position': 'Goalkeeper', 'Age': '26'}
{'Position': 'Attacking Midfield', 'Age': '23'}
{'Position': 'Left-Back', 'Age': '19'}
{'Position': 'Attacking Midfield', 'Age': '35'}
{'Position': 'Centre-Back', 'Age': '19'}
{'Position': 'Right Winger', 'Age': '34'}
{'Position': 'Centre-Forward', 'Age': '24'}
{'Position': 'Left-Back', 'Age': '24'}
{'Position': 'Central Midfield', 'Age': '31'}
{'Position': 'Goalkeeper', 'Age': '23'}
{'Position': 'Central Midfield', 'Age': '30'}
{'Position': 'Attacking Midfield', 'Age': '18'}
{'Position': 'Right Midfield', 'Age': '21'}
{'Position': 'Right-Back', 'Age': '24'}
{'Position': 'Central Midfield', 'Age': '22'}
{'Position': 'Left-Back', 'Age': '27'}
{'Position': 'Central Midfield', 'Age': '21'}
{'Position': 'Centre-Back', 'Age': '24'}
{'Position': 'Defensive Midfield', 'Age': '30'}
{'Position': 'Attacking Midfield', 'Age': '25'}
{'Position': 'Centre-Back', 'Age': '22'}
{'P

{'Position': 'Right-Back', 'Age': '22'}
{'Position': 'Centre-Forward', 'Age': '30'}
{'Position': 'Left-Back', 'Age': '22'}
{'Position': 'Centre-Back', 'Age': '30'}
{'Position': 'Defensive Midfield', 'Age': '30'}
{'Position': 'Goalkeeper', 'Age': '29'}
{'Position': 'Left Winger', 'Age': '30'}
{'Position': 'Left-Back', 'Age': '36'}
{'Position': 'Centre-Forward', 'Age': '27'}
{'Position': 'Centre-Back', 'Age': '28'}
{'Position': 'Centre-Back', 'Age': '29'}
{'Position': 'Left Winger', 'Age': '31'}
{'Position': 'Left Winger', 'Age': '29'}
{'Position': 'Centre-Forward', 'Age': '18'}
{'Position': 'Attacking Midfield', 'Age': '25'}
{'Position': 'Centre-Forward', 'Age': '30'}
{'Position': 'Attacking Midfield', 'Age': '26'}
{'Position': 'Attacking Midfield', 'Age': '25'}
{'Position': 'Right-Back', 'Age': '25'}
{'Position': 'Centre-Back', 'Age': '22'}
{'Position': 'Central Midfield', 'Age': '26'}
{'Position': 'Goalkeeper', 'Age': '30'}
{'Position': 'Central Midfield', 'Age': '19'}
{'Position': 'C

{'Position': 'Right Winger', 'Age': '19'}
{'Position': 'Left Winger', 'Age': '25'}
{'Position': 'Right Winger', 'Age': '27'}
{'Position': 'Right-Back', 'Age': '17'}
{'Position': 'Left-Back', 'Age': '30'}
{'Position': 'Attacking Midfield', 'Age': '24'}
{'Position': 'Attacking Midfield', 'Age': '28'}
{'Position': 'Second Striker', 'Age': '21'}
{'Position': 'Attacking Midfield', 'Age': '24'}
{'Position': 'Centre-Forward', 'Age': '21'}
{'Position': 'Attacking Midfield', 'Age': '24'}
{'Position': 'Defensive Midfield', 'Age': '31'}
{'Position': 'Central Midfield', 'Age': '22'}
{'Position': 'Attacking Midfield', 'Age': '18'}
{'Position': 'Centre-Back', 'Age': '19'}
{'Position': 'Centre-Back', 'Age': '27'}
{'Position': 'Goalkeeper', 'Age': '19'}
{'Position': 'Central Midfield', 'Age': '24'}
{'Position': 'Centre-Back', 'Age': '23'}
{'Position': 'Centre-Back', 'Age': '34'}
{'Position': 'Attacking Midfield', 'Age': '24'}
{'Position': 'Centre-Forward', 'Age': '25'}
{'Position': 'Left Winger', 'Age

{'Position': 'Left Winger', 'Age': '18'}
{'Position': 'Centre-Forward', 'Age': '21'}
{'Position': 'Goalkeeper', 'Age': '19'}
{'Position': 'Left Winger', 'Age': '32'}
{'Position': 'Goalkeeper', 'Age': '31'}
{'Position': 'Left-Back', 'Age': '30'}
{'Position': 'Centre-Forward', 'Age': '26'}
{'Position': 'Left-Back', 'Age': '32'}
{'Position': 'Central Midfield', 'Age': '20'}
{'Position': 'Goalkeeper', 'Age': '38'}
{'Position': 'Centre-Back', 'Age': '24'}
{'Position': 'Central Midfield', 'Age': '23'}
{'Position': 'Central Midfield', 'Age': '19'}
{'Position': 'Central Midfield', 'Age': '25'}
{'Position': 'Defensive Midfield', 'Age': '24'}
{'Position': 'Right Winger', 'Age': '29'}
{'Position': 'Centre-Back', 'Age': '30'}
{'Position': 'Centre-Forward', 'Age': '17'}
{'Position': 'Central Midfield', 'Age': '18'}
{'Position': 'Goalkeeper', 'Age': '38'}
{'Position': 'Centre-Back', 'Age': '28'}
{'Position': 'Right-Back', 'Age': '37'}
{'Position': 'Goalkeeper', 'Age': '21'}
{'Position': 'Centre-Forw

{'Position': 'Centre-Forward', 'Age': '27'}
{'Position': 'Left Winger', 'Age': '26'}
{'Position': 'Defensive Midfield', 'Age': '31'}
{'Position': 'Right Midfield', 'Age': '19'}
{'Position': 'Central Midfield', 'Age': '27'}
{'Position': 'Attacking Midfield', 'Age': '34'}
{'Position': 'Right Winger', 'Age': '22'}
{'Position': 'Centre-Forward', 'Age': '22'}
{'Position': 'Right-Back', 'Age': '24'}
{'Position': 'Centre-Back', 'Age': '32'}
{'Position': 'Centre-Forward', 'Age': '29'}
{'Position': 'Centre-Back', 'Age': '26'}
{'Position': 'Goalkeeper', 'Age': '30'}
{'Position': 'Goalkeeper', 'Age': '22'}
{'Position': 'Goalkeeper', 'Age': '32'}
{'Position': 'Left-Back', 'Age': '27'}
{'Position': 'Attacking Midfield', 'Age': '44'}
{'Position': 'Right-Back', 'Age': '25'}
{'Position': 'Defensive Midfield', 'Age': '31'}
{'Position': 'Goalkeeper', 'Age': '28'}
{'Position': 'Right-Back', 'Age': '22'}
{'Position': 'Right-Back', 'Age': '27'}
{'Position': 'Left-Back', 'Age': '37'}
{'Position': 'Centre-Ba

...

{'Position': 'Centre-Forward', 'Age': '19'}
{'Position': 'Centre-Back', 'Age': '24'}
{'Position': 'Goalkeeper', 'Age': '28'}
{'Position': 'Centre-Forward', 'Age': '24'}
{'Position': 'Left-Back', 'Age': '26'}
{'Position': 'Right Midfield', 'Age': '22'}
{'Position': 'Attacking Midfield', 'Age': '22'}
{'Position': 'Centre-Forward', 'Age': '23'}
{'Position': 'Centre-Forward', 'Age': '32'}
{'Position': 'Centre-Back', 'Age': '23'}
{'Position': 'Attacking Midfield', 'Age': '28'}
{'Position': 'Defensive Midfield', 'Age': '32'}
{'Position': 'Right-Back', 'Age': '22'}
{'Position': 'Centre-Forward', 'Age': '20'}
{'Position': 'Left Winger', 'Age': '19'}
{'Position': 'Centre-Forward', 'Age': '23'}
{'Position': 'Central Midfield', 'Age': '34'}
{'Position': 'Central Midfield', 'Age': '23'}
{'Position': 'Goalkeeper', 'Age': '37'}
{'Position': 'Attacking Midfield', 'Age': '34'}
{'Position': 'Left-Back', 'Age': '27'}
{'Position': 'Goalkeeper', 'Age': '33'}
{'Position': 'Right Winger', 'Age': '26'}
{'Pos

{'Position': 'Central Midfield', 'Age': '23'}
{'Position': 'Centre-Back', 'Age': '29'}
{'Position': 'Left-Back', 'Age': '20'}
{'Position': 'Centre-Forward', 'Age': '34'}
{'Position': 'Right-Back', 'Age': '25'}
{'Position': 'Centre-Forward', 'Age': '28'}
{'Position': 'Right Winger', 'Age': '26'}
{'Position': 'Right Winger', 'Age': '24'}
{'Position': 'Attacking Midfield', 'Age': '32'}
{'Position': 'Centre-Forward', 'Age': '35'}
{'Position': 'Attacking Midfield', 'Age': '21'}
{'Position': 'Centre-Back', 'Age': '29'}
{'Position': 'Left Winger', 'Age': '17'}
{'Position': 'Centre-Forward', 'Age': '25'}
{'Position': 'Centre-Back', 'Age': '32'}
{'Position': 'Central Midfield', 'Age': '20'}
{'Position': 'Central Midfield', 'Age': '23'}
{'Position': 'Central Midfield', 'Age': '16'}
{'Position': 'Defensive Midfield', 'Age': '25'}
{'Position': 'Centre-Back', 'Age': '22'}
{'Position': 'Right Midfield', 'Age': '23'}
{'Position': 'Centre-Back', 'Age': '23'}
{'Position': 'Right-Back', 'Age': '28'}
{'P

{'Position': 'Central Midfield', 'Age': '24'}
{'Position': 'Centre-Back', 'Age': '24'}
{'Position': 'Right Winger', 'Age': '22'}
{'Position': 'Centre-Forward', 'Age': '22'}
{'Position': 'Centre-Forward', 'Age': '24'}
{'Position': 'Left Winger', 'Age': '33'}
{'Position': 'Centre-Forward', 'Age': '29'}
{'Position': 'Right Winger', 'Age': '20'}
{'Position': 'Centre-Back', 'Age': '23'}
{'Position': 'Right Winger', 'Age': '27'}
{'Position': 'Left Winger', 'Age': '26'}
{'Position': 'Left-Back', 'Age': '29'}
{'Position': 'Centre-Forward', 'Age': '24'}
{'Position': 'Attacking Midfield', 'Age': '31'}
{'Position': 'Left Winger', 'Age': '20'}
{'Position': 'Central Midfield', 'Age': '24'}
{'Position': 'Central Midfield', 'Age': '20'}
{'Position': 'Right-Back', 'Age': '21'}
{'Position': 'Central Midfield', 'Age': '21'}
{'Position': 'Left Winger', 'Age': '24'}
{'Position': 'Right Winger', 'Age': '19'}
{'Position': 'Central Midfield', 'Age': '24'}
{'Position': 'Right-Back', 'Age': '19'}
{'Position': 

{'Position': 'Centre-Back', 'Age': '24'}
{'Position': 'Right-Back', 'Age': '23'}
{'Position': 'Centre-Back', 'Age': '22'}
{'Position': 'Goalkeeper', 'Age': '20'}
{'Position': 'Left Winger', 'Age': '27'}
{'Position': 'Central Midfield', 'Age': '30'}
{'Position': 'Centre-Forward', 'Age': '33'}
{'Position': 'Centre-Forward', 'Age': '19'}
{'Position': 'Defensive Midfield', 'Age': '22'}
{'Position': 'Centre-Back', 'Age': '22'}
{'Position': 'Centre-Back', 'Age': '27'}
{'Position': 'Defensive Midfield', 'Age': '22'}
{'Position': 'Central Midfield', 'Age': '18'}
{'Position': 'Second Striker', 'Age': '22'}
{'Position': 'Left-Back', 'Age': '22'}
{'Position': 'Centre-Forward', 'Age': '20'}
{'Position': 'Left Midfield', 'Age': '23'}
{'Position': 'Centre-Forward', 'Age': '21'}
{'Position': 'Central Midfield', 'Age': '21'}
{'Position': 'Left-Back', 'Age': '22'}
{'Position': 'Right Winger', 'Age': '20'}
{'Position': 'Centre-Back', 'Age': '28'}
{'Position': 'Right-Back', 'Age': '25'}
{'Position': 'Def

{'Position': 'Central Midfield', 'Age': '26'}
{'Position': 'Goalkeeper', 'Age': '28'}
{'Position': 'Left Winger', 'Age': '20'}
{'Position': 'Centre-Back', 'Age': '29'}
{'Position': 'Right-Back', 'Age': '26'}
{'Position': 'Right Winger', 'Age': '22'}
{'Position': 'Defensive Midfield', 'Age': '29'}
{'Position': 'Right-Back', 'Age': '17'}
{'Position': 'attack', 'Age': '18'}
{'Position': 'Centre-Back', 'Age': '20'}
{'Position': 'Central Midfield', 'Age': '21'}
{'Position': 'Centre-Back', 'Age': '18'}
{'Position': 'Centre-Back', 'Age': '28'}
{'Position': 'Defensive Midfield', 'Age': '33'}
{'Position': 'Right Winger', 'Age': '33'}
{'Position': 'Centre-Forward', 'Age': '30'}
{'Position': 'Centre-Back', 'Age': '30'}
{'Position': 'Goalkeeper', 'Age': '40'}
{'Position': 'Centre-Forward', 'Age': '21'}
{'Position': 'Centre-Back', 'Age': '33'}
{'Position': 'Centre-Back', 'Age': '16'}
{'Position': 'Centre-Forward', 'Age': '31'}
{'Position': 'Goalkeeper', 'Age': '26'}
{'Position': 'Goalkeeper', 'Age'

In [139]:
print(len(extra_attr_lst))
print(type(extra_attr_lst))
my_df

6300
<class 'list'>


,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,...,Launch%,Average Pass Length,Goal Kicks,Launch% (Goal Kicks),Avg. Length of Goal Kicks,Crosses Faced,Crosses Stopped,Crosses Stopped %,Def. Actions Outside Pen. Area,Avg. Distance of Def. Actions
0,Aaron Connolly,0.19,0.0,0.19,0.00,0.00,0.00,0.00,0.54,0.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Cresswell,0.02,0.2,0.02,0.00,0.00,0.07,0.00,0.03,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AarÃ³n Escandell,NaN,NaN,NaN,NaN,0.22,NaN,NaN,NaN,NaN,...,69.4%,49.7,8.63,81.0%,59.2,8.85,0.66,7.4%,2.29,16.8
3,Aaron Herzog,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aaron Hickey,0.21,0.0,0.21,0.00,0.00,0.28,0.07,0.03,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,Zinho Vanheusden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6296,Zlatan IbrahimoviÄ,0.57,0.1,0.52,0.05,0.10,0.16,0.05,0.65,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6297,Zlatko JunuzoviÄ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6298,Zouhair Feddal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
#my_df = my_df.append(extra_attr_lst, ignore_index=True,  sort=False)
extra_df = pd.DataFrame(extra_attr_lst)
extra_df.to_csv(r'extra_df.csv')
extra_df

,Position,Age
0,Centre-Forward,21
1,Left-Back,31
2,Goalkeeper,26
3,Attacking Midfield,23
4,Left-Back,19
...,...,...
6295,Centre-Back,22
6296,Centre-Forward,40
6297,Central Midfield,34
6298,Centre-Back,31


In [176]:
result_df = pd.concat([extra_df,my_df], axis=1)

array(['Centre-Forward', 'Left-Back', 'Goalkeeper', 'Attacking Midfield',
       'Centre-Back', 'Right Winger', 'Central Midfield',
       'Right Midfield', 'Right-Back', 'Defensive Midfield',
       'Left Winger', 'Second Striker', 'attack', 'Left Midfield',
       'midfield', 'Defender'], dtype=object)

# Data cleaning (Drop goalkeeper and NaN value)

In [177]:
drop_col = result_df.columns.to_list()[138:]
drop_col

['Goals Against',
 'Shots on Target Against',
 'Saves',
 'Save%',
 'Wins',
 'Draws',
 'Losses',
 'Clean Sheets',
 'Clean Sheet Percentage',
 'Penalty Kicks Allowed',
 'Penalty Kicks Saved',
 'Penalty Kicks Missed',
 'Save% (Penalty Kicks)',
 'Free Kick Goals Against',
 'Corner Kick Goals Against',
 'Own Goals Scored Against Goalkeeper',
 'PSxG',
 'PSxG/SoT',
 'PSxG-GA',
 'Passes Completed (Launched)',
 'Passes Attempted (Launched)',
 'Pass Completion Percentage (Launched)',
 'Throws Attempted',
 'Launch%',
 'Average Pass Length',
 'Goal Kicks',
 'Launch% (Goal Kicks)',
 'Avg. Length of Goal Kicks',
 'Crosses Faced',
 'Crosses Stopped',
 'Crosses Stopped %',
 'Def. Actions Outside Pen. Area',
 'Avg. Distance of Def. Actions']

In [180]:
result_df=result_df.drop(columns=drop_col)
result_df.to_csv(r'dataset_v1.csv')
result_df

,Position,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,...,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
0,Centre-Forward,21,Aaron Connolly,0.19,0.0,0.19,0.00,0.00,0.00,0.00,...,2.30,0.19,0.0,0.0,0.00,2.87,0.96,3.26,22.7%,€7.00
1,Left-Back,31,Aaron Cresswell,0.02,0.2,0.02,0.00,0.00,0.07,0.00,...,0.44,0.02,0.0,0.0,0.00,8.49,1.16,0.69,62.7%,€5.00
2,Goalkeeper,26,AarÃ³n Escandell,NaN,NaN,NaN,NaN,0.22,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€1.00
3,Attacking Midfield,23,Aaron Herzog,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"€175Th. Last update: Oct 11, 2021"
4,Left-Back,19,Aaron Hickey,0.21,0.0,0.21,0.00,0.00,0.28,0.07,...,2.23,0.00,0.0,0.0,0.00,5.99,0.63,0.84,42.9%,€8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,Centre-Back,22,Zinho Vanheusden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€12.00
6296,Centre-Forward,40,Zlatan IbrahimoviÄ,0.57,0.1,0.52,0.05,0.10,0.16,0.05,...,1.36,2.51,0.1,0.0,0.05,3.76,3.60,2.66,57.5%,€4.00
6297,Central Midfield,34,Zlatko JunuzoviÄ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€2.50
6298,Centre-Back,31,Zouhair Feddal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€6.00


In [181]:
sum([True for idx,row in result_df.iterrows() if any(row.isnull())])

4406

In [185]:
result_df=result_df.dropna()
result_df = result_df.reset_index()
result_df

,index,Position,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,...,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
0,0,Centre-Forward,21,Aaron Connolly,0.19,0.00,0.19,0.00,0.00,0.00,...,2.30,0.19,0.00,0.0,0.00,2.87,0.96,3.26,22.7%,€7.00
1,1,Left-Back,31,Aaron Cresswell,0.02,0.20,0.02,0.00,0.00,0.07,...,0.44,0.02,0.00,0.0,0.00,8.49,1.16,0.69,62.7%,€5.00
2,4,Left-Back,19,Aaron Hickey,0.21,0.00,0.21,0.00,0.00,0.28,...,2.23,0.00,0.00,0.0,0.00,5.99,0.63,0.84,42.9%,€8.00
3,8,Centre-Forward,24,Aaron Leya Iseka,0.31,0.00,0.21,0.10,0.21,0.42,...,1.14,0.21,0.00,0.0,0.00,3.12,2.60,3.43,43.1%,€2.20
4,9,Left-Back,24,AarÃ³n MartÃ­n,0.00,0.11,0.00,0.00,0.00,0.11,...,0.76,0.11,0.00,0.0,0.00,6.35,0.73,1.27,36.4%,€4.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,6276,Central Midfield,25,Yves Bissouma,0.00,0.03,0.00,0.00,0.00,0.23,...,1.20,0.00,0.00,0.0,0.00,10.22,0.88,0.60,59.6%,€25.00
1890,6286,Central Midfield,22,Zaydou Youssouf,0.07,0.07,0.07,0.00,0.00,0.20,...,2.07,0.13,0.00,0.0,0.07,8.67,0.87,0.87,50.0%,€5.00
1891,6289,Right-Back,24,Zeki Ãelik,0.05,0.08,0.05,0.00,0.00,0.29,...,0.53,0.08,0.03,0.0,0.00,7.64,1.22,0.93,56.8%,€20.00
1892,6292,Right Midfield,28,Zinedine Ferhat,0.13,0.31,0.09,0.04,0.04,0.09,...,2.01,0.35,0.09,0.0,0.00,5.34,1.49,1.88,44.2%,€5.50


In [186]:
sum([True for idx,row in result_df.iterrows() if any(row.isnull())])

0

In [187]:
result_df.to_csv(r'dataset_v2.csv')

# Start to preprocess

In [190]:
print(result_df)

index                 int64
Position             object
Age                  object
Name                 object
Goals               float64
                     ...   
Ball Recoveries     float64
Aerials won         float64
Aerials lost        float64
% of Aerials Won     object
Market value         object
Length: 139, dtype: object


In [200]:
loaded_csv = pd.read_csv(r'dataset_v2.csv',index_col=[0])
loaded_csv=loaded_csv.drop(["index"], axis=1)
loaded_csv

,Position,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,...,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
0,Centre-Forward,21,Aaron Connolly,0.19,0.00,0.19,0.00,0.00,0.00,0.00,...,2.30,0.19,0.00,0.0,0.00,2.87,0.96,3.26,22.7%,€7.00
1,Left-Back,31,Aaron Cresswell,0.02,0.20,0.02,0.00,0.00,0.07,0.00,...,0.44,0.02,0.00,0.0,0.00,8.49,1.16,0.69,62.7%,€5.00
2,Left-Back,19,Aaron Hickey,0.21,0.00,0.21,0.00,0.00,0.28,0.07,...,2.23,0.00,0.00,0.0,0.00,5.99,0.63,0.84,42.9%,€8.00
3,Centre-Forward,24,Aaron Leya Iseka,0.31,0.00,0.21,0.10,0.21,0.42,0.00,...,1.14,0.21,0.00,0.0,0.00,3.12,2.60,3.43,43.1%,€2.20
4,Left-Back,24,AarÃ³n MartÃ­n,0.00,0.11,0.00,0.00,0.00,0.11,0.00,...,0.76,0.11,0.00,0.0,0.00,6.35,0.73,1.27,36.4%,€4.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,Central Midfield,25,Yves Bissouma,0.00,0.03,0.00,0.00,0.00,0.23,0.00,...,1.20,0.00,0.00,0.0,0.00,10.22,0.88,0.60,59.6%,€25.00
1890,Central Midfield,22,Zaydou Youssouf,0.07,0.07,0.07,0.00,0.00,0.20,0.07,...,2.07,0.13,0.00,0.0,0.07,8.67,0.87,0.87,50.0%,€5.00
1891,Right-Back,24,Zeki Ãelik,0.05,0.08,0.05,0.00,0.00,0.29,0.00,...,0.53,0.08,0.03,0.0,0.00,7.64,1.22,0.93,56.8%,€20.00
1892,Right Midfield,28,Zinedine Ferhat,0.13,0.31,0.09,0.04,0.04,0.09,0.00,...,2.01,0.35,0.09,0.0,0.00,5.34,1.49,1.88,44.2%,€5.50
